In [1]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

cap = cv2.VideoCapture(0)
count = 0

while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 200: 
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-1-9e7bfa020eb6>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [2]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

Labels = np.asarray(Labels, dtype=np.int32)


lucky_model  = cv2.face_LBPHFaceRecognizer.create()
lucky_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


Model trained sucessefully


In [4]:
import cv2
import numpy as np
import os
import boto3
import smtplib, ssl
import pywhatkit as pw
import time



tm = time.localtime()
time_hour = tm.tm_hour
time_min = tm.tm_min

session = 0
counting = 0

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi



cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        results = lucky_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident you are Lucky!!'
            
        #cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 90:
            cv2.putText(image, "Hey Lucky!!!!", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            
            session+=1
            if (session == 50):
                pw.sendwhatmsg("+919348567136","Hello sonu!!!!",time_hour,time_min + 1)
                port = 465
                smtp_server = "smtp.gmail.com"
                sender_email = "laxmannaik5055@gmail.com"
                password = input(str("What is your password for mail:"))
                receiver_email = "laxman211naik@gmail.com"
                message = "this is face of Laxman_Naik!!!!"

                context = ssl.create_default_context()
                with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
                    server.login( sender_email , password )
                    server.sendmail( sender_email , receiver_email , message)
                    print("Mail and Message sent successfully!!!!")
                    break
            
            
            
        else:
            
            cv2.putText(image, "I dont know, who r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            counting+=1
            if (counting == 50):
                os.system("aws ec2 run-instances --image-id ami-0ad704c126371a549 --instance-type t2.micro --security-group-ids sg-0d84c7546c2a14e47  --count 1 --key-name ec2key")
                print("instance launched")
                x = int(input("index of instance to be used: "))
                instance_id = boto3.client("ec2").describe_instances()['Reservations'][x]['Instances'][0]['InstanceId']
                zone = boto3.client("ec2").describe_instances()['Reservations'][x]['Instances'][0]['Placement']['AvailabilityZone']
                os.system("aws ec2 create-volume --volume-type gp2 --size 5 --availability-zone "+ zone)
                print("EBS launched")
                y = int(input("index of volume to be used: "))
                vol_id = boto3.client("ec2").describe_volumes()['Volumes'][1]['VolumeId']
                os.system("aws ec2 attach-volume --volume-id "+ vol_id +" --instance-id " + instance_id + " --device /dev/xvdf")
                print("attached")
                break
            
            
            
            
    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()     

<>:24: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:24: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-4-78f598b5becf>:24: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


instance launched
index of instance to be used: 1
EBS launched
index of volume to be used: 0
attached


In [ ]:
cap.release()
    

In [ ]:
Thank you.....